In [184]:
import pandas as pd

file = "Resources/purchase_data.csv"
data = pd.read_csv(file, header = 0)
data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [185]:
#Counting all unique player SN's
players = data['SN'].nunique()
print(players)

576


In [186]:
#Creating a dictionary with our number of players

total_players = {"Total Players": players}
tp = pd.DataFrame(total_players, index = ['0'])
tp.head()

,Total Players
0,576


In [187]:
#Number of Unique Items

items = data['Item ID'].nunique()
print(items)

183


In [188]:
#Average Price per item

average_price = data['Price'].mean()
average_price

3.050987179487176

In [189]:
#Number of Purchases

purchases = data.shape[0]
purchases

780

In [190]:
#Total Revenue

revenue = data['Price'].sum()
revenue

2379.77

In [191]:
#Creating 'Player Analysis' DataFrame with desired calculations

player_analysis = {"Number of Unique Items": items, "Average Price": average_price, "Number of Purchases": purchases, "Total Revenue": revenue}
pa = pd.DataFrame(player_analysis, index = ['0'])

#Formatting Currencies
pa['Average Price'] = pa['Average Price'].map('${:,.2f}'.format)
pa['Total Revenue'] = pa['Total Revenue'].map('${:,.2f}'.format)
pa.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [192]:
#Getting Gender counts

#Grouping original dataframe by "SN", then counting the number of unique 
grouped = data.groupby("Gender")["SN"].nunique()

#Collecting desired counts from the new grouped dataframe
female_count = grouped.loc['Female']
male_count = grouped.loc['Male']
other_count = grouped.loc['Other / Non-Disclosed']

grouped.head()


Gender
Female                    81
Male                     484
Other / Non-Disclosed     11
Name: SN, dtype: int64

In [193]:
#Calculating Percentages for each Gender

male_percentage = (male_count/players)*100
female_percentage = (female_count/players)*100
other_percentage = (other_count/players)*100

In [194]:
#Creating 'Gender Demographics' dataframe

gender_dem = {"Total Count": [male_count, female_count, other_count], "Percentages": [male_percentage, female_percentage, other_percentage]}
gd = pd.DataFrame(gender_dem, index = ["Male", "Female", "Other / Non-Disclosed"])
gd['Percentages'] = gd['Percentages'].map('%{:,.2f}'.format)
gd

,Total Count,Percentages
Male,484,%84.03
Female,81,%14.06
Other / Non-Disclosed,11,%1.91


In [195]:
#Collecting 'Purchasing Analysis (Gender)' Information

#Grouping original dataframe by gender, then 'Purchase Count' variables
gender_group = data.groupby("Gender").size()

female_pc = gender_group.loc["Female"]
male_pc = gender_group.loc["Male"]
other_pc = gender_group.loc["Other / Non-Disclosed"]

gender_group

Gender
Female                   113
Male                     652
Other / Non-Disclosed     15
dtype: int64

In [196]:
#Grouping original dataframe by 'Gender', then summing up 'Price' values
av_price = data.groupby("Gender")["Price"].sum()

#Calculating respective average purchase prices
female_ap = av_price.loc["Female"]/female_pc
male_ap = av_price.loc["Male"]/male_pc
other_ap = av_price.loc["Other / Non-Disclosed"]/other_pc

#Gathering 'Total Purchase Value' by Gender
female_pv = av_price.loc["Female"]
male_pv = av_price.loc["Male"]
other_pv = av_price.loc["Other / Non-Disclosed"]

av_price

Gender
Female                    361.94
Male                     1967.64
Other / Non-Disclosed      50.19
Name: Price, dtype: float64

In [197]:
#Average Total Purchase per Person

#Grouping original dataframe by gender and sn, then summing the price each person spent, reset index to allow regrouping by gender to get the mean for each gender
tppp = data.groupby(["Gender", "SN"])["Price"].sum().reset_index().groupby("Gender").mean()

female_tppp = tppp.iloc[0,0]
male_tppp = tppp.iloc[1,0]
other_tppp = tppp.iloc[2,0]
tppp.head()


,Price
Gender,
Female,4.468395
Male,4.065372
Other / Non-Disclosed,4.562727


In [198]:
#Creating 'Purchasing Analysis (Gender)' dataframe

pur_an_g = {"Purchase Count": [female_pc, male_pc, other_pc], "Average Purchase Price": [female_ap, male_ap, other_ap], 
            "Total Purchase Value": [female_pv, male_pv, other_pv], "Avg Total Purchase per Person": [female_tppp, male_tppp, other_tppp]}
pag = pd.DataFrame(pur_an_g, index = ["Female", "Male", "Other / Non-Disclosed"])
pag['Average Purchase Price'] = pag['Average Purchase Price'].map('${:,.2f}'.format)
pag['Total Purchase Value'] = pag['Total Purchase Value'].map('${:,.2f}'.format)
pag['Avg Total Purchase per Person'] = pag['Avg Total Purchase per Person'].map('${:,.2f}'.format)
pag

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [203]:
#Collecting info for 'Age Demographics'

#Organzing original dataframe by age range
age_bins = [-1, 9, 14, 19, 24, 29, 34, 39, 100]
bin_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
data["Age Range"] = pd.cut(data["Age"], age_bins, labels = bin_names)

#Creating new dataframe that counts unique values for each age range
data2 = data.groupby(["Age Range", "SN"])["Age"].count().reset_index().groupby("Age Range").count()

#Gathering counts and calculating percentages for each age range
age_counts = []
age_count_df = data2["Age"]
for x in age_count_df:
    age_counts.append(x)
    
age_percentages = []    
for y in age_counts:
    perc = (y/players)*100
    age_percentages.append(perc)

data2

,SN,Age
Age Range,,
<10,17,17
10-14,22,22
15-19,107,107
20-24,258,258
25-29,77,77
30-34,52,52
35-39,31,31
40+,12,12


In [206]:
#Creating 'Age Demographics' DataFrame
age_demo = {"Total Count": age_counts, "Percentage of Players": age_percentages}
ad = pd.DataFrame(age_demo, index = [bin_names])
ad['Percentage of Players'] = ad['Percentage of Players'].map('%{:,.2f}'.format)
ad

,Total Count,Percentage of Players
<10,17,%2.95
10-14,22,%3.82
15-19,107,%18.58
20-24,258,%44.79
25-29,77,%13.37
30-34,52,%9.03
35-39,31,%5.38
40+,12,%2.08


In [211]:
#Purchasing Analysis (Age)

#Getting Purchase Counts per age group
purchase_counts = []
data3 = data.groupby("Age Range").count().reset_index()
purchase_count_df = data3["Purchase ID"]
for x in purchase_count_df:
    purchase_counts.append(x)
purchase_counts

[23, 28, 136, 365, 101, 73, 41, 13]

In [255]:
#Collecting Average Purchase Price per Age Group
data4 = data.groupby("Age Range")["Price"].mean()
average_purch = []
for x in data4:
    average_purch.append(x)

#Collecting Total Purchase Price per Age Group
data4 = data.groupby("Age Range")["Price"].sum()
total_purch = []
for y in data4:
    total_purch.append(y)

#Collecting Average Purchase Price per Person, per Age Group    
data4 = data.groupby(["Age Range", "SN"])["Price"].sum().reset_index().groupby("Age Range").mean()
av_tppp = []
av_tppp_df = data4["Price"]
for row in av_tppp_df:
    av_tppp.append(row)
av_tppp

[4.537058823529413,
 3.7627272727272727,
 3.85878504672897,
 4.318062015503876,
 3.8051948051948052,
 4.115384615384615,
 4.763548387096775,
 3.186666666666666]

In [257]:
#Creating 'Purchasing Analysis (age)' DataFrame
purch_an_age = {"Purchase Count": purchase_counts, "Average Purchase Price": average_purch, "Total Purchase Value": total_purch, 
               "Average Purchase Price per Person": av_tppp}
paa = pd.DataFrame(purch_an_age, index = [bin_names])
paa['Average Purchase Price'] = paa['Average Purchase Price'].map('${:,.2f}'.format)
paa['Total Purchase Value'] = paa['Total Purchase Value'].map('${:,.2f}'.format)
paa['Average Purchase Price per Person'] = paa['Average Purchase Price per Person'].map('${:,.2f}'.format)
paa

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Price per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [301]:
#Gathering data for 'Top Spenders' DataFrame

#Purchase Count
data5 = data.groupby("SN").count().reset_index()
names = []
names_df = data5["SN"]
for x in names_df:
    names.append(x)
sn_purch_count = []
sn_df = data5["Purchase ID"]
for y in sn_df:
    sn_purch_count.append(y)

#Average Purchase Price
data5 = data.groupby("SN")["Price"].mean().reset_index()
av_pp = []
av_pp_df = data5["Price"]
for z in av_pp_df:
    av_pp.append(z)

#Total Purchase Value
data5 = data.groupby("SN")["Price"].sum().reset_index()
total_pv = []
total_pv_df = data5["Price"]
for z in total_pv_df:
    total_pv.append(z)
len(total_pv)

576

In [309]:
#Creating 'Top Spenders' DataFrame

top_spenders = {"SN": names, "Purchase Count": sn_purch_count, "Average Purchase Price": av_pp, "Total Purchase Value": total_pv}
ts_df = pd.DataFrame(top_spenders)
# ts_df['Average Purchase Price'] = ts_df['Average Purchase Price'].map('${:,.2f}'.format)
# ts_df['Total Purchase Value'] = ts_df['Total Purchase Value'].map('${:,.2f}'.format)
ts_df.sort_values(by = ["Total Purchase Value"], inplace = True, ascending = False)
ts_df.head()

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
360,Lisosia93,5,3.792000,18.96
246,Idastidru52,4,3.862500,15.45
106,Chamjask73,3,4.610000,13.83
275,Iral74,4,3.405000,13.62
281,Iskadarya95,3,4.366667,13.10
